## First, MANUALLY create a copy of the data file (the script will modify the data file)

In [1]:
import numpy as np
import xarray as xr
import sys
import glob
import netCDF4 as nc
import os
import sys
import h5py
import scipy.io
from scipy.interpolate import griddata
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm
import cartopy.feature as cfeature
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as colors

from os.path import join,expanduser,exists,split
user_home_dir = expanduser('~')
sys.path.append(join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

from functions_ecco_NEW import shift_longitude, remove_text_after_second_underscore, calculate_data2plot, \
map_contributor_positive_negative, map_first_or_third_contributor, map_second_contributor, plot_map_from_scattered

# Suppress warning messages for a cleaner presentation
import warnings
warnings.filterwarnings('ignore')

In [2]:
# year_start = 2004  # Define the start year
# year_end = 2016  # Define the end year (NOT included, will stop the year before this one)

year_start = 1992  # Define the start year
year_end = 2018  # Define the end year (NOT included, will stop the year before this one)

# ECCO MONTHLY and OISST
# year_start = 1993  # Define the start year
# year_end = 2016  # Define the end year (NOT included, will stop the year before this one)

# Select the level
lev_or_int = 'zlev00' 
# lev_or_int = 'zlev01' # JUST FOR ECCO MONTHLY 
# lev_or_int = 'zlev05' 
# lev_or_int = 'zlev09' 
# lev_or_int = 'ohc_k0_k5'
# lev_or_int = 'oisst'
# lev_or_int = 'argo'

dataset_tag = 'ECCOv4r4_heat' # argo_ohc15_50 oisst_v2 ECCOv4r4_heat
daily_monthly_plot_tag = 'daily' # 'daily', 'monthly'
ecco_monthly_tag_field = 'heat' # JUST FOR ECCO MONTHLY OHC (ohc_to50m) vs LEVELS (or daily ohc) (heat)
ecco_monthly_tag = False # True False
ecco_daily_tag = True # False
test_peaks_tag = False
delta_time_tag = '5tsteps' # '5tsteps' 1 for OISST and ECCO monthly (zlev01), 5 for daily
test_data_alignment = False
year_subsel_tag = True

# MHW duration thresholds
MHW_duration_min_threshold = 1 # days (new minimum duration for MHW events) # 5, 10, 30, 60
MHW_duration_max_threshold = np.inf # days (new maximum duration for MHW events) # 10, 30, 60, np.inf
MHW_duration_threshold_tag = str(MHW_duration_min_threshold) + '_' + str(MHW_duration_max_threshold) + '_days'

# load_dir = '/Users/jacoposala/Downloads/figures_Nov3_k9/daily/'
load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_daily_temp_zlev0_1992_2018/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_daily_temp_zlev0_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/ECCOv4r4_daily_temp_zlev5_1992_2018/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/ECCOv4r4_daily_temp_zlev5_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/ECCOv4r4_daily_temp_zlev9_1992_2018/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/ECCOv4r4_daily_temp_zlev9_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_daily_temp_ohc_k0_k5_1992_2018/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_daily_temp_ohc_k0_k5_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/OISST_1993_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/OISST_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_monthly_temp_zlev0_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_monthly_temp_zlev0_1993_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/Argo_2004_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/peaks_test_ECCOv4r4_daily_temp_ohc_k0_k5_1992_2018/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_monthly_temp_ohc_to50m_1993_2016/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/ECCOv4r4_monthly_temp_ohc_to50m_1993_2016_k0_k5/output/figures/'
# load_dir = '/Users/jacoposala/Downloads/Blanca_Outputs/ECCOv4r4_monthly_temp_ohc_to50m_2004_2016_k0_k5/output/figures/'

ECCO_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/NEW_heatBudgetECCO/data/'
save_dir = join(ECCO_dir,'outputs')
path = '/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/NEW_heatBudgetECCO/data/outputs/nc_files_zlev_or_zint/'
save_path = '/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/NEW_heatBudgetECCO/code/summary_maps/' + lev_or_int + '/'

# add plot_type_tag = ['all', 'min60days', 'min30days'] etc (include also new percentiles (95%))


In [3]:
from datetime import datetime

# Create datetime objects for the start and end dates
start_date = datetime(year_start, 1, 1)
end_date = datetime(year_end-1, 12, 31)

# Calculate the difference between the two dates
difference = end_date - start_date

# Get the total number of days
total_days = difference.days

In [4]:
# ECCO
if dataset_tag == 'ECCOv4r4_heat':
    # Load ECCO grid data
    XC_lon = xr.open_dataset(path + 'ECCOv4r4_XC_lon_1993_2017.nc')
    YC_lat = xr.open_dataset(path + 'ECCOv4r4_YC_lat_1993_2017.nc')
    Z_depth = xr.open_dataset(path + 'ECCOv4r4_Z_depth_1993_2017.nc')
    # ECCO daily files
    if ecco_daily_tag:
        # OHC
        if lev_or_int == 'ohc_k0_k5':
            file_path = load_dir + f'ECCOv4r4_heat_daily_{year_start}_{year_end}_prcnt90_noTrend_minLen_5tsteps_withAVE.mat'
        # Heat budget terms
        else:
            file_path = load_dir + f'ECCOv4r4_heat_{lev_or_int}_daily_{year_start}_{year_end}_prcnt90_noTrend_minLen_5tsteps_withAVE.mat'
    # ECCO monthly files
    if ecco_monthly_tag:
        if ecco_monthly_tag_field == 'ohc_to50m':
            file_path = load_dir + f'ECCOv4r4_{ecco_monthly_tag_field}_{lev_or_int}_{year_start}_{year_end}_prcnt90_noTrend_minLen_1tsteps_withAVE.mat'
        else:
            file_path = load_dir + f'ECCOv4r4_heat_{lev_or_int}_{year_start}_{year_end}_prcnt90_noTrend_minLen_1tsteps_withAVE.mat'
    
    # Open the HDF5 file with write permissions
    with h5py.File(file_path, 'a') as f:
        # Access the data under the key 'find_MHWs_info'
        find_mhws_info_data = f['find_MHWs_info']
        # Now you can modify the dataset
        for key in find_mhws_info_data.keys():
            if find_mhws_info_data[key].value.shape[0] == total_days-1:
                # Remove the first 366 elements (year 1992) and the last 365 elements (years 2017 - note that 2018 is not included to begin with)
                sliced_subset = find_mhws_info_data[key][365:-365, :, :]
                # Resize the original dataset to match the shape of the sliced subset
                find_mhws_info_data[key].resize(sliced_subset.shape)
                # Overwrite the existing dataset with the modified subset
                find_mhws_info_data[key][...] = sliced_subset

                print(key)
                print(sliced_subset.shape)


G_advection_declineAve
(8400, 13, 8100)
G_advection_eventAve
(8400, 13, 8100)
G_advection_onsetAve
(8400, 13, 8100)
G_diffusion_declineAve
(8400, 13, 8100)
G_diffusion_eventAve
(8400, 13, 8100)
G_diffusion_onsetAve
(8400, 13, 8100)
G_forcing_declineAve
(8400, 13, 8100)
G_forcing_eventAve
(8400, 13, 8100)
G_forcing_onsetAve
(8400, 13, 8100)
G_total_declineAve
(8400, 13, 8100)
G_total_eventAve
(8400, 13, 8100)
G_total_onsetAve
(8400, 13, 8100)
adv_vConv_declineAve
(8400, 13, 8100)
adv_vConv_eventAve
(8400, 13, 8100)
adv_vConv_onsetAve
(8400, 13, 8100)
data_mhw_tstep_msk
(8400, 13, 8100)
data_percentile3d
(8400, 13, 8100)
data_used4MHWs
(8400, 13, 8100)
data_used4MHWs_declineAve
(8400, 13, 8100)
data_used4MHWs_eventAve
(8400, 13, 8100)
data_used4MHWs_onsetAve
(8400, 13, 8100)
dif_vConv_declineAve
(8400, 13, 8100)
dif_vConv_eventAve
(8400, 13, 8100)
dif_vConv_onsetAve
(8400, 13, 8100)
end_tstep
(8400, 13, 8100)
end_tstep_stored_at_peak
(8400, 13, 8100)
events_duration_in_tsteps
(8400, 13, 

## NOW, need to manually change the name of the modified file in the Finder to reflect the new years

In [ ]:
# Load the data selected above
mat_data = h5py.File(file_path, 'r')

# List the keys in the file
# print("Keys in the file:", list(mat_data.keys()))

# Access the data under the key '#refs#'
refs_data = mat_data['#refs#']

# Access the data under the key 'find_MHWs_info'
find_mhws_info_data = mat_data['find_MHWs_info']

In [ ]:
find_mhws_info_data['G_advection_declineAve'].value.shape